In [2]:
cd ../scheduler

/Users/gcallah/GitProjects/Scheduler/scheduler


# Welcome to Scheduler!

### Getting started
To begin run the cell below (shift + enter) to create the sample data we need. This will take data from datamaker.py (which is the file  where you can add/edit/remove classes, teachers, classrooms; and exports that data into a file called sample_data.txt to be used.

In [3]:
import json
import datamaker

datamaker.make_data()

Now we load the created data into our program (run the cell below)

In [4]:
input_file = "sample_data.txt"
with open(input_file,'r') as f:
    data = json.load(f)
    
professors = data['professors']
prof_info = data['prof_info']
rooms = data['rooms']
room_capacities = data['room_capacities']
courses = data['courses']
course_no_students = data['course_no_students']
course_mins = data['course_mins']
course_days_weekly = data['course_days_weekly']

The data is ready to use. 

Running the cell below shows us a list of available professors

In [5]:
print('\n'.join(professors))

John Smith
Lisa Jones
Mike Williams
Tim Simpson
Rachel Smith
Gregg Woods
Simon Valinski
Chu Yen
Peter Parker
Lisa Mullen
Elizabeth Walker
Brian K. Dickson
Jamir Abdullah


Similarly, run the cell below to see a list of rooms

In [6]:
print('\n'.join(rooms))

655
666
745a
745b
433
201
115a
115b


And finally do the same for the list of courses

In [7]:
print('\n'.join(courses))

physics
chemistry
biochemistry
biology 1
biology 2
molecular biology
calculus 1
calculus 4
astrophysics


The scheduler program will take the courses shown above and find a professor, classroom and time for them.

### Scheduling the Teachers, Courses and Classrooms 

Run the following two cells

In [8]:
from teachercourse_csp import assigner

In [9]:
def time_formatter(course,start_time):
    hs,ms = start_time
    end = hs*6 + ms//10 + course_mins[course]//10
    end_time = (end//6,(end - (end//6)*6)*10)
    he,me = end_time
    output = '{:0>2}'.format(hs)+':'+'{:0>2}'.format(ms)+' to ' + '{:0>2}'.format(he)+':'+ '{:0>2}'.format(me) 
    return output

Now we are ready to compute the class schedule and display it below


In [10]:
full_schedule = assigner()
weekdays = ['mon','tues','wed','thur','fri']
for day in weekdays:
    schedule = full_schedule[day]
    print(day.upper())
    print('-----')
    for var,val in schedule.items():
        course,professor = var
        room,start_time = val
        print("%s - %s - %s - %s" % (course, professor, room, time_formatter(course,start_time)))
    print('')

MON
-----
physics - Lisa Jones - 745b - 14:00 to 15:00
chemistry - John Smith - 745b - 13:00 to 14:30
biology 1 - Peter Parker - 745b - 08:00 to 09:30
calculus 1 - Simon Valinski - 115b - 12:00 to 13:00
calculus 4 - Chu Yen - 433 - 14:00 to 15:00

TUES
-----
physics - Lisa Jones - 115b - 13:30 to 14:30
biology 2 - Peter Parker - 745b - 10:30 to 11:30
molecular biology - Peter Parker - 433 - 12:00 to 13:00
astrophysics - Simon Valinski - 655 - 12:00 to 13:00

WED
-----
physics - Lisa Jones - 115a - 10:30 to 11:30
chemistry - John Smith - 115b - 15:30 to 17:00
biology 1 - Peter Parker - 115b - 11:00 to 12:30
biology 2 - Peter Parker - 115b - 12:30 to 13:30
calculus 1 - Simon Valinski - 115a - 12:30 to 13:30

THUR
-----
biochemistry - Gregg Woods - 115b - 13:00 to 14:30
biology 2 - Peter Parker - 433 - 11:00 to 12:00

FRI
-----
biochemistry - Gregg Woods - 666 - 13:30 to 15:00
calculus 1 - Simon Valinski - 745b - 15:30 to 16:30
calculus 4 - Chu Yen - 115b - 17:00 to 18:00

